In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold


In [2]:
df = pd.read_csv('../dataset/Auto.csv')
df['horsepower'] =df['horsepower'].replace('?',0)
df['horsepower'] =df['horsepower'].astype(int)
y = df['displacement']

del df['name']

In [3]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


In [4]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower        int64
weight            int64
acceleration    float64
year              int64
origin            int64
dtype: object

In [5]:
#split it into train and test 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(df, y,test_size = 0.33 , random_state = 1)

X_train = pd.DataFrame(X_train, columns = df.columns)
X_test = pd.DataFrame(X_test, columns = df.columns)


In [7]:
#Create dataset of lightgbm

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

#get number of feature and number of training rows
num_train, num_feature = X_train.shape

#Get parameters 
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}


#------------------------------------------------------------
folds = KFold(n_splits=5, shuffle=True, random_state=123)

oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])

feature_name = [col for col in X_train.columns]
print(feature_name)
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    trn_x, trn_y = X_train[feature_name].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = X_train[feature_name].iloc[val_idx], y.iloc[val_idx]
    

#Generate feature name 


print('Starting training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                valid_sets=lgb_train,  # eval training data
                feature_name=feature_name,
                categorical_feature=[21])

['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']
Starting training...
[1]	training's rmse: 96.6611
[2]	training's rmse: 91.9543
[3]	training's rmse: 87.4897
[4]	training's rmse: 83.2595
[5]	training's rmse: 79.2301
[6]	training's rmse: 75.4454
[7]	training's rmse: 71.847
[8]	training's rmse: 68.4427
[9]	training's rmse: 65.1488
[10]	training's rmse: 62.0257


/home/tung/miniconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [21]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
